In [20]:
import sys
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_timestamp, to_date, lit

# Khởi tạo Spark session
spark = SparkSession.builder.appName('CrimeAnalysis').getOrCreate()

# Đọc dữ liệu từ Parquet
df = spark.read.parquet("C:/Users/ADMIN/DATA ANALYST/Projectdata/Chicago_Crimes_ZIP.parquet")

# Nhập ngày bắt đầu và kết thúc
start_date = '03/15/2018'
end_date = '03/15/2019'

# Chuyển đổi cột 'Date' thành timestamp
df = df.withColumn('timestamp', to_timestamp(col('Date'), 'MM/dd/yyyy hh:mm:ss a'))

# Chuyển đổi start_date và end_date thành định dạng date
start_date_parsed = to_date(lit(start_date), 'MM/dd/yyyy')
end_date_parsed = to_date(lit(end_date), 'MM/dd/yyyy')

# Lọc dữ liệu theo khoảng thời gian
filtered_df = df.filter((col('timestamp').between(start_date_parsed, end_date_parsed)))

# Nhóm theo loại tội phạm (PrimaryType) và đếm số lượng
crime_count_df = filtered_df.groupBy('PrimaryType').count()

# Ghi kết quả vào tệp CSV
crime_count_df.coalesce(1).write.mode("overwrite").csv('C:/Users/ADMIN/DATA ANALYST/Projectdata/CrimeTypeCount', header=True)

# Đọc lại dữ liệu từ thư mục đã ghi vào
import os
import pandas as pd

# Tìm tệp CSV trong thư mục được tạo bởi PySpark (
csv_path = 'C:/Users/ADMIN/DATA ANALYST/Projectdata/CrimeTypeCount'

# Kiểm tra các tệp trong thư mục
csv_files = [os.path.join(csv_path, file) for file in os.listdir(csv_path) if file.endswith('.csv')]
crime_count_pd = pd.read_csv(csv_files[0])

# Vẽ biểu đồ thanh sử dụng matplotlib
import matplotlib.pyplot as plt

# Vẽ biểu đồ thanh
plt.figure(figsize=(10, 6))
plt.bar(crime_count_pd['PrimaryType'], crime_count_pd['count'])
plt.xlabel('Crime Type')
plt.ylabel('Count')
plt.title('Chicago Crime Count by Type')
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()

# Dừng Spark session
spark.stop()
